In [9]:
import numpy as np
import matplotlib.pyplot as plt
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5EncoderModel
from torch import nn
import torch
from torch.nn import functional as F
from torch import optim
from torch.utils.data import DataLoader
from Datasets import Modes, WikiTableText
from PIL import Image

In [10]:
# tokenizer = T5Tokenizer.from_pretrained('t5-base')

In [11]:
# pergunta = "what was the last year where this team was a part of the usl a-league?"
# tokenizer = T5Tokenizer.from_pretrained('t5-base')
# target = tokenizer.encode_plus(pergunta,
#                                        padding='max_length',
#                                        truncation=True,
#                                        max_length=128,
#                                        return_tensors='pt')

In [12]:
# encoder = T5EncoderModel.from_pretrained('t5-base')
# decoder = decoder = T5ForConditionalGeneration.from_pretrained('t5-base')

In [13]:
# decoder.config.d_ff

In [14]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
ds = WikiTableText(Modes.TRAIN, tokenizer=tokenizer, max_len=1024)
sample = ds[5]

In [15]:
sample

{'input_ids': tensor([  822,    10,    84,  ...,  3180, 19563,     1]),
 'input_attn_mask': tensor([1, 1, 1,  ..., 1, 1, 1]),
 'question': 'which is deeper, lake tuz or lake palas tuzla?',
 'answer': 'Lake Palas Tuzla',
 'table_str': '| Name in English | Name in Turkish | Area (km2) | Depth | Location (districts and/or provinces) |\n| Lake Van | Van Gölü | 3755\xa0km2 | 451\xa0m | Van, Bitlis |\n| Lake Tuz | Tuz Gölü | 1500\xa0km2 | 2\xa0m | Aksaray, Ankara, Konya |\n| Lake Beyşehir | Beyşehir Gölü | 656\xa0km2 | 10\xa0m | Beyşehir in Konya, Isparta |\n| Lake Eğirdir | Eğirdir Gölü | 482\xa0km2 | | Isparta |\n| Lake İznik | İznik Gölü | 308\xa0km2 | | İznik in Bursa, Yalova |\n| Lake Burdur | Burdur Gölü | 200\xa0km2 | | Burdur, Isparta |\n| Lake Manyas | Manyas Gölü | 166\xa0km2 | | Balıkesir |\n| Lake Acıgöl | Acıgöl | 153\xa0km2 | | Denizli, Afyonkarahisar |\n| Lake Uluabat | Uluabat Gölü | 134\xa0km2 | 1–2\xa0m | Bursa |\n| Lake Çıldır | Çıldır Gölü | 115\xa0km2 | | Ardahan, Kars |

In [16]:
import itertools
def sample():
    
    decoder = T5ForConditionalGeneration.from_pretrained('t5-base')
#     encoder = T5EncoderModel.from_pretrained('t5-base')

    encoder = decoder.get_encoder()
    
    dl = DataLoader(ds, batch_size=15, shuffle=True)
    samples = next(iter(dl))
        
    opt = optim.Adam(itertools.chain(encoder.parameters(), decoder.parameters()), lr=5e-4)
    
    for epo in range(10):
        opt.zero_grad()
        
        encoder.train()
        decoder.train()
        
        
        enc_out = encoder(input_ids=samples['input_ids'],
                          attention_mask=samples['input_attn_mask'],
                          output_attentions=True)
        
        hidden = enc_out.last_hidden_state
        
        loss = decoder(encoder_outputs=(hidden, ), 
                       labels=samples['target_ids']).loss
        
        print(f"Vai pfv loss: {loss}")
        loss.backward()
        opt.step()
        
        with torch.no_grad():
            encoder.eval()
            decoder.eval()
            
            enc_out = encoder(input_ids=samples['input_ids'],
                     attention_mask=samples['input_attn_mask'],
                             output_attentions=True)
        
            encoder_hidden_states = enc_out.last_hidden_state
            

            B = encoder_hidden_states.size(0)
    
            decoded_ids = torch.full((B, 1),
                                 decoder.config.decoder_start_token_id,
                                 dtype=torch.long)
            
            for step in range(20):
                outputs = decoder(decoder_input_ids=decoded_ids,
                                  encoder_outputs=(encoder_hidden_states,),
                                  return_dict=True)
#                 outputs = decoder(decover_input_embeds)
                logits = outputs["logits"]

                next_token_logits = logits[:, -1, :]

                # Greedy decoding
                next_token_id = next_token_logits.argmax(1).unsqueeze(-1)

                # Check if output is end of senquence for all batches
                if torch.eq(next_token_id[:, -1], tokenizer.eos_token_id).all():
                    break

                # Concatenate past ids with new id, keeping batch dimension
                decoded_ids = torch.cat([decoded_ids, next_token_id], dim=-1)
            
            print(f"{epo} - Decoded: {tokenizer.batch_decode(decoded_ids, skip_special_tokens=True)}")
            print(f"{epo} - Real: {samples['answer']}")
    
    print(f"{epo} - Question: {samples['question']}")
    print(f"{epo} - Question: {samples['table_str']}")
#     for elem in enumerate(samples['table_img']):
#         img = np.transpose(elem.cpu().numpy(), (1, 2, 0))
#         plt.figure(figsize=(15, 15))
#         plt.imshow(img)
#         plt.show()
#         print(f"Question: {samples['question'][idx]}")
#     print(f"Questions: {samples['question']}")
    del encoder, decoder, hidden, enc_out, opt

sample()

Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Vai pfv loss: 4.6575140953063965
0 - Decoded: ['6,859', 'Chantelle and Steve', 'Los Angeles Rams', 'Foolad ad 5-2 on penalties 5-2 on penalties', 'United States s s s s s s s', '27 Jul', 'Chicago', '20,000', '24', '2', '7', 'Eskaton 001', 'SGD 2,400', 'H. C. McNeile', '2 s s s s s s s']
0 - Real: ['6,859', 'Brad and Dale', 'Los Angeles Rams', 'Foolad|Esteghlal Ahvaz', 'Canada|United States|Austria|Italy', '07 Aug', 'Buffalo', '45', '6', '7', '6', 'Coil', 'Ravi Sandiran', 'H. C. McNeile', '2']
Vai pfv loss: 1.6400213241577148
1 - Decoded: ['6,859', 'Brad and Dale', 'Los Angeles Rams', 'Fooladghlal Ahvaz', 'Canada', '04 Aug', 'Chicago', '72', '24', '2', '7', 'Coil', 'Ravi Sandiran', 'H. C. McNeile', '2']
1 - Real: ['6,859', 'Brad and Dale', 'Los Angeles Rams', 'Foolad|Esteghlal Ahvaz', 'Canada|United States|Austria|Italy', '07 Aug', 'Buffalo', '45', '6', '7', '6', 'Coil', 'Ravi Sandiran', 'H. C. McNeile', '2']
Vai pfv loss: 1.0090794563293457
2 - Decoded: ['6,859', 'Brad and Dale', 'Los 

UnboundLocalError: local variable 'features' referenced before assignment

##### target.input_ids.shape, target.attention_mask.shape

In [ ]:
decoder.get_encoder()

In [ ]:
a = encoder(input_ids=target.input_ids, attention_mask=target.attention_mask, output_attentions=True)

In [ ]:
a.last_hidden_state

In [ ]:
a

In [ ]:
v.shape, a.last_hidden_state.shape

In [ ]:
test = torch.cat([v.view(1, -1),a.last_hidden_state.view(1, -1)], dim=1)

In [ ]:
test.shape

In [ ]:
labels = tokenizer.encode_plus("204",
                                       padding='max_length',
                                       truncation=True,
                                       max_length=128,
                                       return_tensors='pt').input_ids

In [ ]:
labels.shape

In [ ]:
decoder = T5ForConditionalGeneration.from_pretrained('t5-base')

In [ ]:
decoder(encoder_outputs=(t.permute(0, 2, 1), ), labels=labels).loss

In [ ]:
decoder.config.d_model

In [ ]:
ajustado = embedding_extractor(img_tensor.unsqueeze(0)).permute(0, 2, 3, 1)

In [ ]:
ajustado = ajustado.view(1, -1, 768)

In [ ]:
ajustado = ajustado.view(1, -1, 768)

In [ ]:
ajustado.shape

In [ ]:
ajustado.permute(0, 2, 1).shape

In [ ]:
l = nn.Linear(1575, 128)

In [ ]:
t = l(ajustado.permute(0, 2, 1))

In [ ]:
t.permute(0, 2, 1).shape

In [ ]:
p = '/WikiTableQuestions/csv/200-csv/1.table'
f = open(p, 'r')
x = f.read()

In [ ]:
# x = x.replace("\t", " \t ")
# x = x.replace("\"", "")

In [ ]:
import re
x = re.sub("\s\s+" , " ", x)
x = x.replace("\n", " \n ")
# x.replace(/  +/g, ' ')

In [ ]:
tokenizer.decode(tokenizer.encode(x))

In [ ]:
x